In [ ]:
# Standard libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Import functions from your qualitative_analysis package
from qualitative_analysis.data_processing import load_data, clean_and_normalize, sanitize_dataframe
from qualitative_analysis.prompt_construction import construct_prompt
from qualitative_analysis.model_interaction import get_llm_client
from qualitative_analysis.response_parsing import extract_code_from_response
from qualitative_analysis.evaluation import compute_cohens_kappa
from qualitative_analysis.utils import save_results_to_csv, load_results_from_csv
import qualitative_analysis.config as config

# Additional libraries
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Define data directory
data_dir = 'data'  # Adjust the path as needed
os.makedirs(data_dir, exist_ok=True)

In [ ]:
# Define the path to your dataset
data_file_path = os.path.join(data_dir, 'datasets', 'divergent_questions_data', 'qa_data_RE.csv')

# Load the data
data = load_data(data_file_path, file_type='csv', delimiter=',')  # Adjust delimiter if needed

# Preview the data
data.head()

In [ ]:
# Define text columns to clean
text_columns = ['text', 'questions']  # Replace with your actual column names

# Clean and normalize text columns
for col in text_columns:
    data[col] = clean_and_normalize(data[col])

# Sanitize the DataFrame
data = sanitize_dataframe(data)

In [ ]:
# Combine texts and questions
data['verbatim'] = data.apply(lambda row: f"Text: {row['text']}\n\nQuestion: {row['questions']}", axis=1)

# Extract the list of verbatims
verbatims = data['verbatim'].tolist()

print(f"Total number of verbatims: {len(verbatims)}")
print(f"Verbatim example:\n{verbatims[0]}")

In [ ]:
from qualitative_analysis.prompt_construction import construct_prompt as base_construct_prompt

def construct_prompt(verbatim):
    # Your custom prompt construction logic
    codebook = """
    Instructions for classifying the question based on the text:

    - 0: The answer to the question is explicitly stated in the text.
    - 1: The answer to the question is not explicitly stated but can be implied from the text.
    - 2: The answer to the question is not at all stated in the text.
    """

    examples = """
    [Include your examples here as in your original prompt]
    """

    instructions = "You are a helpful assistant tasked with classifying questions with respect to a given text."

    # Build the prompt using the base_construct_prompt function
    prompt = base_construct_prompt(
        data_format_description="",
        entry_text=verbatim,
        codebook=codebook,
        examples=examples,
        instructions=instructions,
        selected_fields=['Score'],
        output_format_example={'Score': '0'}
    )

    return prompt

In [ ]:
# Choose the provider and model
provider = 'together'  # or 'azure' if using OpenAI
model_name = 'meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo'  # Adjust as needed

# Initialize the client
llm_client = get_llm_client(provider=provider, config=config.MODEL_CONFIG[provider])

In [ ]:
results = []
total_tokens_used = 0

for idx, verbatim in enumerate(verbatims):
    print(f"Processing Verbatim {idx+1}/{len(verbatims)}")
    prompt = construct_prompt(verbatim)
    
    try:
        response = llm_client.get_response(
            prompt=prompt,
            model=model_name,
            max_tokens=500,
            temperature=0.0001,
            verbose=False
        )
        # Parse the response using the backend function
        score = extract_code_from_response(response)
        if score is not None:
            results.append({
                'Verbatim': verbatim,
                'Score': score
            })
        else:
            print(f"Failed to parse response for Verbatim {idx+1}")
            results.append({
                'Verbatim': verbatim,
                'Score': None
            })
    except Exception as e:
        print(f"Error processing Verbatim {idx+1}: {e}")
        results.append({
            'Verbatim': verbatim,
            'Score': None
        })

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Define the save path
outputs_dir = os.path.join(data_dir, 'outputs')
os.makedirs(outputs_dir, exist_ok=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_path = os.path.join(outputs_dir, f"experiment_{model_name.replace('/', '_')}_{timestamp}.csv")

# Save results
save_results_to_csv(
    coding=results_df.to_dict('records'),
    save_path=save_path,
    fieldnames=['Verbatim', 'Score'],
    verbatims=None  # Verbatims are included in the results
)

print(f"Results saved to: {save_path}")

In [ ]:
# Load results
loaded_results = load_results_from_csv(save_path)
# The function returns (verbatims, coding)
verbatims_loaded, coding_loaded = loaded_results

In [ ]:
# Assuming you have human annotations in the data
human_annotations = data['div_rater1'].tolist()  # Replace with actual column name
model_coding = results_df['Score'].tolist()

# Compute Cohen's Kappa
kappa = compute_cohens_kappa(
    human_annotations,
    model_coding,
    labels=[0, 1, 2],
    weights='linear'
)

print(f"Cohen's Kappa Score between human annotations and model: {kappa:.2f}")

In [ ]:
# Confusion Matrix
cm = confusion_matrix(human_annotations, model_coding, labels=[0, 1, 2])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1, 2])
disp.plot()
plt.show()